# Purpose of this notebook

Do something useful with the  [relational data model](https://opendata.tweedekamer.nl/documentatie/informatiemodel) that the tweede kamer provides -- see also 
[extras_datacollect_tweede_kamer_dossiers](extras_datacollect_tweede_kamer_dossiers.ipynb).


Our quest is "make a list of Who is part of what party.

This involves four different resources: Persoon, FractieZetelPersoon, FractieZetel, Fractie

You can see FractieZetelPersoon is a many-to-many joiner table
- a reference to a FractieZetel, which itself just references a Fractie
- a reference to a and Persoon

Persoon and Fractie have no references so are just details to each item.

The below code does relational joins for party membership, 
then flattens the results by adding membership to the persoon details.

In [1]:
import json
import pprint

import wetsuite.datacollect.tweedekamer_nl   # contains some basic code dealing with this API
import wetsuite.helpers.etree

In [2]:
# Fetch all resources of the mentioned soort.
# the combination should take a minute or two.
print("Fetching Persoon")
data_Persoon            = wetsuite.datacollect.tweedekamer_nl.merge_etrees( wetsuite.datacollect.tweedekamer_nl.fetch_all( 'Persoon' ) )
print("Fetching Fractie")
data_Fractie            = wetsuite.datacollect.tweedekamer_nl.merge_etrees( wetsuite.datacollect.tweedekamer_nl.fetch_all( 'Fractie' ) )
print("Fetching FractieZetel")
data_FractieZetel       = wetsuite.datacollect.tweedekamer_nl.merge_etrees( wetsuite.datacollect.tweedekamer_nl.fetch_all( 'FractieZetel' ) )
print("Fetching FractieZetelPersoon")
data_FractieZetelPesoon = wetsuite.datacollect.tweedekamer_nl.merge_etrees( wetsuite.datacollect.tweedekamer_nl.fetch_all( 'FractieZetelPersoon' ) )

Fetching Persoon
Fetching Fractie
Fetching FractieZetel
Fetching FractieZetelPersoon


In [4]:
# makes it slightly easier to fetch individual items in later code:
#   split into items, let us look up by id, and by type
id_thing = {}   #               guid -> detailsdict
by_type  = {}   #     soort/category -> list of all such detailsdicts

for etree in (data_Persoon, data_Fractie, data_FractieZetel, data_FractieZetelPesoon):
    for entry_dict in wetsuite.datacollect.tweedekamer_nl.entry_dicts( etree ):
        id_thing[ entry_dict['id'] ] = entry_dict

        category = entry_dict['category']
        if category not in by_type:
            by_type[category] = []
        by_type[ category ].append( entry_dict )

In [5]:
# This is basically a manual JOIN

persoon_combined = {}  # which is mostly still the persoon dicts, except we added a key that is the membership

for fzp_dict in by_type['fractieZetelPersoon']:
    if 'fractieZetel' not in fzp_dict:
        print("  FractieZetelPersoon item without FractieZetel - huh? %r"%fzp_dict)
        continue

    _, fractiezetel_id = fzp_dict['fractieZetel']
    _, persoon_id      = fzp_dict['persoon']

    fractiezetel_dict = id_thing[ fractiezetel_id   ]
    persoon_dict      = id_thing[ persoon_id ]

    _, fractie_id = fractiezetel_dict['fractie']
    frac_dict = id_thing[ fractie_id ]

    if persoon_id not in persoon_combined:
        persoon_combined[ persoon_id ] = persoon_dict
        persoon_combined[ persoon_id ]['fractie_membership'] = []
        
    persoon_combined[ persoon_id ]['fractie_membership'].append( 
        {
            'fractie_id':frac_dict['id'], 
            'fractie_afkorting':frac_dict['afkorting'], 
            # maybe more or all of frac_dict?
            'functie':fzp_dict['functie'], 
            'van':fzp_dict['van'], 
            'totEnMet':fzp_dict['totEnMet'],
        } 
    )


  FractieZetelPersoon item without FractieZetel - huh? {'title': 'd73d7f69-1235-4746-aa94-84b593909bfc', 'updated': '2023-08-29T14:00:09Z', 'category': 'fractieZetelPersoon', 'id': 'd73d7f69-1235-4746-aa94-84b593909bfc'}
  FractieZetelPersoon item without FractieZetel - huh? {'title': '8cca26af-365a-46fc-b72f-c42b2a17a992', 'updated': '2023-12-06T13:48:37Z', 'category': 'fractieZetelPersoon', 'id': '8cca26af-365a-46fc-b72f-c42b2a17a992'}
  FractieZetelPersoon item without FractieZetel - huh? {'title': '6d685347-a5c8-498b-94df-b7bbb1bc1272', 'updated': '2023-12-06T13:48:37Z', 'category': 'fractieZetelPersoon', 'id': '6d685347-a5c8-498b-94df-b7bbb1bc1272'}


In [6]:

print("WRITING tweedekamer-fractie-membership-struc.json")
with open('tweedekamer-fractie-membership-struc.json','wb') as jsonfile:
    jsonfile.write( json.dumps( persoon_combined ).encode('ascii') )

print("WRITING tweedekamer-fracties-struc.json")
fracs = {} # id -> details, for consistency with the above and easier lookup
for fracs_dict in by_type['fractie']:
    fracs[ fracs_dict['id'] ] = fracs_dict

with open('tweedekamer-fracties-struc.json','wb') as jsonfile:
    jsonfile.write( json.dumps( fracs ).encode('ascii') )


WRITING tweedekamer-fractie-membership-struc.json
WRITING tweedekamer-fracties-struc.json
